In [3]:
from mlflow.tracking import MlflowClient

MlflowClient_URI = "sqlite:///mlflow.db"


client=  MlflowClient(tracking_uri=MlflowClient_URI)

In [7]:
client.search_experiments()

[<Experiment: artifact_location='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/1', creation_time=1730815586467, experiment_id='1', last_update_time=1730815586467, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/0', creation_time=1730815586435, experiment_id='0', last_update_time=1730815586435, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
client.create_experiment(name='my-cool-experiment')

'2'

In [10]:
# finding the best model in an experiment
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='1',
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [11]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.318445793399953}, params={'learning_rate': '0.09585355369315604',
  'max_depth': '30',
  'min_child_weight': '1.060597050922164',
  'objective': 'reg:linear',
  'reg_alpha': '0.018060244040060163',
  'reg_lambda': '0.011658731377413597',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "77af998e8ddd49cea6962e12d833966d", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2024-11-05 '
                              '15:11:30.872384", "model_uuid": '
                              '"d3bd1a70fdc5460195458ecd7bbda5df", "flavors": '
                              '{"python_function": {"loader_module": '
                              '"mlflow.xgboost", "python_version": "3.12.4", '
                              '"data": "model.xgb", "env": {"conda": '
                              '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                              '"xgboost

In [12]:
for run in runs:
    print(f"run id {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id 77af998e8ddd49cea6962e12d833966d, rmse: 6.3184
run id 3b8eed8966ee494b924f04fd63f275ed, rmse: 6.3505
run id 26f11eb8585d4dbeaae120d2c042d66b, rmse: 6.5821
run id 77d9227279a24cbf991bf3825f6685f5, rmse: 12.1432
run id 58be6e83a1dd4e53832de01e4247d3a3, rmse: 12.1432


In [13]:
## moving some runs to the registry

In [15]:
import mlflow

mlflow.set_tracking_uri(MlflowClient_URI)

run_id = "77d9227279a24cbf991bf3825f6685f5"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri,name="nyc-taxi-records")


Registered model 'nyc-taxi-records' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-records'.


<ModelVersion: aliases=[], creation_timestamp=1730821597445, current_stage='None', description=None, last_updated_timestamp=1730821597445, name='nyc-taxi-records', run_id='77d9227279a24cbf991bf3825f6685f5', run_link=None, source='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/1/77d9227279a24cbf991bf3825f6685f5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [16]:
model_uri

'runs:/77d9227279a24cbf991bf3825f6685f5/model'

In [17]:
# move the model to staging

In [18]:
client.list_registed_models()

AttributeError: 'MlflowClient' object has no attribute 'list_registed_models'

In [22]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1730819669820, description='This is the Nyc Taxi Predictor for trip duration', last_updated_timestamp=1730821597445, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1730819950027, current_stage='Staging', description='', last_updated_timestamp=1730820315332, name='nyc-taxi-records', run_id='3b8eed8966ee494b924f04fd63f275ed', run_link='', source='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/1/3b8eed8966ee494b924f04fd63f275ed/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1730821597445, current_stage='None', description=None, last_updated_timestamp=1730821597445, name='nyc-taxi-records', run_id='77d9227279a24cbf991bf3825f6685f5', run_link=None, source='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/1/77d9227279a24cbf991bf3825f6685f5/art

In [24]:
model_name = "nyc-taxi-records"
latest_versions= client.get_latest_versions(name=model_name)

/tmp/ipykernel_51238/3854861217.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions= client.get_latest_versions(name=model_name)


In [27]:
for version in latest_versions:
    print(f"version {version.version} {version.current_stage}")

version 2 Staging
version 3 None


In [30]:
client.transition_model_version_stage(
    name=model_name,
    version =3,
    archive_existing_versions=False,
    stage="Staging"

)

/tmp/ipykernel_51238/2789967756.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1730821597445, current_stage='Staging', description=None, last_updated_timestamp=1730822463952, name='nyc-taxi-records', run_id='77d9227279a24cbf991bf3825f6685f5', run_link=None, source='/home/akogo/Desktop/MLOps/002-experinment-tracking/002-training/experinment-Tracking/mlruns/1/77d9227279a24cbf991bf3825f6685f5/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>